Spinner with Threads Examples

In [2]:
import itertools
import time
from threading import Thread, Event

def spin(msg:str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

def supervisor() -> int:
    done = Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

def main() -> None:
    result = supervisor()
    print(f'Answer: {result}')

main()

Spinner With Processes Examples

In [3]:
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize

def spin(msg: str, done:synchronize.Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

def main() -> None:
    result = supervisor()
    print(f'Answer: {result}')

main()

Spinner with Coroutines Examples

In [6]:
# ****** DOESN'T RUN IN JUPYTER NOTEBOOK ******
# import asyncio
# import itertools


# async def spin(msg: str) -> None:
#     for char in itertools.cycle(r'\|/-'):
#         status = f'\r{char} {msg}'
#         print(status, flush=True, end='')
#         try:
#             await asyncio.sleep(.1)
#         except asyncio.CancelledError:
#             break
#         blanks = ' ' * len(status)
#         print(f'\r{blanks}\r', end='')

# async def slow() -> int:
#     await asyncio.sleep(3)
#     return 42

# def main() -> None:
#     result = asyncio.run(supervisor())
#     print(f'Answer: {result}')

# async def supervisor() -> int:
#     spinner = asyncio.create_task(spin('thinking!'))
#     print(f'spinner object: {spinner}')
#     result = await slow()
#     spinner.cancel()
#     return result


# main()

Home Grown Processing Examples

In [11]:
# No Special Processing/Threading/Asynchronous Support
from time import perf_counter
from typing import NamedTuple
import math
import random

NUMBERS = random.sample(range(1, (10 ** 16) - 1), 20)
NUMBERS.sort()

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

class Result(NamedTuple):
    prime: bool
    elapsed: float

def check(n: int) -> Result:
    t0 = perf_counter()
    prime = is_prime(n)
    return Result(prime, perf_counter() - t0)

def main() -> None:
    print(f'Checking {len(NUMBERS)} numbers sequentially:')
    t0 = perf_counter()
    for n in NUMBERS:
        prime, elapsed = check(n)
        label = 'P' if prime else ' '
        print(f'{n:16}  {label} {elapsed:9.6f}s')
    
    elapsed = perf_counter() - t0
    print(f'Total time: {elapsed:.2f}s')

main()

Checking 20 numbers sequentially:
 806197576468488     0.000001s
1100325896112989     0.000003s
1444471243904336     0.000000s
2079452573491790     0.000000s
2157013321541349     0.000001s
2932292243945857     0.000001s
3382465896313341     0.000000s
3919958068945177     0.000000s
4423991309363044     0.000000s
5106334516987124     0.000000s
5697599982044049     0.000000s
5917564225388141     0.000001s
6023897150355946     0.000000s
6152883158770469     0.000000s
6326981309015953     0.000001s
7343921675766343     0.000002s
7428173640324217     0.000000s
8070971634250581     0.000000s
9742791749440129     0.000001s
9886942952009141     0.000001s
Total time: 0.00s


In [59]:
# Processing Support

import sys
from time import perf_counter
from typing import NamedTuple
from multiprocessing import Process, SimpleQueue, cpu_count
from multiprocessing import queues

NUMBERS = random.sample(range(1, (10 ** 16) - 1), 20)
NUMBERS.sort()

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

class PrimeResult(NamedTuple):
    n: int
    prime: bool
    elapsed: float

JobQueue = queues.SimpleQueue[int]
ResultQueue = queues.SimpleQueue[PrimeResult]

def check(n: int) -> PrimeResult:
    t0 = perf_counter()
    res = is_prime(n)
    return PrimeResult(n, res, perf_counter() - t0)

def worker(jobs: JobQueue, results: ResultQueue) -> None:
    while n:=- jobs.get():
        results.put(check(n))
    results.put(PrimeResult(0, False, 0.0))

def start_jobs(
    procs: int, jobs: JobQueue, results: ResultQueue
    ) -> None:
    for n in NUMBERS:
        jobs.put(n)
    for _ in range(procs):
        proc = Process(target=worker, args=(jobs, results))
        proc.start()
        jobs.put(0)
        
def main() -> None:
    if len(sys.argv) < 3:
        procs = cpu_count()
    else:
        procs = int(sys.argv[1])
    
    print(f'Checking {len(NUMBERS)} numbers with {procs} processes:')
    t0 = perf_counter()
    jobs: JobQueue = SimpleQueue()
    results: ResultQueue = SimpleQueue()
    start_jobs(procs, jobs, results)
    checked = report(procs, results)
    elapsed = perf_counter() - t0
    
    print(f'{checked} checks in {elapsed:.2f}s')

def report(procs: int, results: ResultQueue) -> int:
    checked = 0
    procs_done = 0
    while procs_done < procs:
        n, prime, elapsed = results.get()
        if n == 0:
            procs_done += 1
        else:
            checked += 1
            label = 'P' if prime else ' '
            print(f'{n:16}  {label} {elapsed:9.6f}s')
    return checked

main()

Checking 20 numbers with 12 processes:
  -3632312480093     0.000008s
 -74973661940626     0.000001s
-338319858084318     0.000000s
-718931892235701     0.000000s
-1427023916407962     0.000000s
-2878000292366306     0.000000s
-3195998622591881     0.000000s
-3558382071680303     0.000000s
-4418097871224570     0.000000s
-4944315729363401     0.000000s
-5134540134611782     0.000000s
-5889956209198187     0.000000s
-6621090084467146     0.000000s
-7159552092076679     0.000001s
-7593764223836840     0.000000s
-7702491246387721     0.000000s
-8464437752253431     0.000000s
-8952712121138857     0.000000s
-9580239678440033     0.000000s
-9706676820581098     0.000000s
20 checks in 0.04s
